---
title: "Presentación 5: Redes Neuronales"
subtitle: "*Machine Learning II*"
author: "Jose Alexander Fuentes Montoya Ph.D(c)"
format:
  revealjs:
    theme: "moon"
    slideNumber: true
    transition: "convex"            # Transición más dinámica
    center: false
    incremental: true
    plugins: ["notes", "highlight", "math", "search", "zoom"]  # Plugins para interactividad
    backgroundTransition: "fade"
title-slide-attributes:
  data-background-image: "logo.png"
  data-background-size: "10%"
  data-background-position: "90% 10%"
---


# Redes Neuronales

# Visión general de la lección

-  **Tensores en aprendizaje profundo**  
-  **Tensores y PyTorch**  
-  **Vectores, matrices y broadcasting**  
-  **Convenciones de notación para redes neuronales**  
-  **Capa totalmente conectada (lineal) en PyTorch**  


# Tensores en aprendizaje profundo


## Vectores, matrices y tensores — Convenciones de notación

::: columns

::: {.column width="33%" style="font-size:0.6em"}
*Rango 0 y 1*

**Escalar**  
(rank‑0 tensor)  
$$x \in \mathbb{R}$$ 

**Vector**  
(rank‑1 tensor)  
$$x \in \mathbb{R}^n,\quad x\in\mathbb{R}^{n\times1}$$  
$$
x = \begin{bmatrix}
x_1\\
x_2\\
\vdots\\
x_n
\end{bmatrix}
$$ 

:::

::: {.column width="33%" style="font-size:0.5em"}
*Rango 2*

**Matriz**  
(rank‑2 tensor)  
$$X \in \mathbb{R}^{m\times n}$$  
$$
X = \begin{bmatrix}
x_{1,1} & x_{1,2} & \dots  & x_{1,n} \\
x_{2,1} & x_{2,2} & \dots  & x_{2,n} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_{m,1} & x_{m,2} & \dots  & x_{m,n}
\end{bmatrix}
$$
:::
:::::

<div style="font-size:80%;">

## Vectores, matrices y tensores — Convenciones de notación

$X$ para referirnos a la “matriz de diseño”. Es decir, la matriz que contiene los ejemplos de entrenamiento y las características (entradas).** 

**$$X \in \mathbb{R}^{n\times m}$$** 

**Y asumiremos la siguiente estructura**  
_(donde $n$ suele emplearse para referirse al número de ejemplos)_ 

$$
X = \begin{bmatrix}
x^{(1)}_{1} & x^{(1)}_{2} & \dots  & x^{(1)}_{m} \\[6pt]
x^{(2)}_{1} & x^{(2)}_{2} & \dots  & x^{(2)}_{m} \\[6pt]
\vdots      & \vdots      & \ddots & \vdots      \\[6pt]
x^{(n)}_{1} & x^{(n)}_{2} & \dots  & x^{(n)}_{m}
\end{bmatrix}
$$
</div>

## Tensor 3D

**$X \in \mathbb{R}^{m\times n\times p}$** 

Se suele pensar como $p$ “rebanadas” de tamaño $m\times n$:  

![Figura 5-1. Tensor 3D](presentacion_files/41.png)

## Tensor 3D imagen

![Tensor 3D ](presentacion_files/42.png)

- **Imagen de un solo canal**  
- **Tensor 3D**: alto × ancho × canales  
- Usado para almacenamiento de arreglos multidimensionales y cómputo paralelo  
- Seguimos aplicando operaciones de vectores y matrices  

<div style="font-size:70%;">
## Ejemplo tensor 4D

![Lote de imágenes (CIFAR-10)](presentacion_files/43.png){width="150%"}


- **Tensor 4D**:  
  `batch_size` × `alto` × `ancho` × `canales`

- Uso: almacenamiento de arreglos multidimensionales  
  y cómputo paralelo, aplicando operaciones de vectores  
  y matrices de manera eficiente.

</div>

## En el contexto de TensorFlow, NumPy y PyTorch

-  **Tensores = arreglos multidimensionales**
-  **La dimensionalidad coincide con el número de índices de** `.shape`


In [ ]:
import torch
import numpy as np
import tensorflow as tf

## 2. tanh

::::: {.columns style="font-size:80%;"}
::: {.column width="40%"}
![Figura 3-9. Función tanh](presentacion_files/27.png)
:::

::: {.column width="60%"}

**Definición**\
$$ f(x) = \frac{e^x - e^{-x}}{\,e^x + e^{-x}\,} $$

-   Rango ((-1,1)).\
-   *Zero-centered*.\
-   También padece *vanishing gradient*.

**Nota de importancia**\
La función *tanh* se introdujo como **alternativa a la Sigmoide** para que la salida se centre en torno a 0, lo cual en muchos casos facilita el entrenamiento y la convergencia.
:::
:::::

## 3. ReLU (Rectified Linear Unit)

::::: {.columns style="font-size:80%;"}
::: {.column width="60%"}
**Definición**\
$$ f(x) = \max(0, x) $$

-   Eficiente, con salida en $[0,\infty)$.\
-   Minimiza *vanishing gradient*.\
-   Problema: si (x\<0), salida = 0 ⇒ “neurona muerta”; no está acotada arriba ⇒ posible *exploding gradient*.

**Nota de importancia**\
La ReLU revolucionó el **entrenamiento de redes profundas** por su bajo costo computacional y su efectividad en la región positiva.
:::

::: {.column width="40%"}
![Figura 3-10. Función ReLU](presentacion_files/28.png)
:::
:::::

## 4. Leaky ReLU

::::: {.columns style="font-size:80%;"}
::: {.column width="40%"}
**Definición**\
$$ f(x) = 
  \begin{cases} 
    x, & x \ge 0 \\ 
    \alpha x, & x < 0 
  \end{cases}$$

-   Evita el problema de neuronas muertas (al permitir un gradiente distinto de 0 en la región negativa).\
-   Mantiene un gradiente continuo para $x < 0$.
:::

::: {.column width="60%"}
-   Pierde parte de la “sparsidad” (pues ya no pone en cero todos los valores negativos).

    **Nota de importancia**\
    La Leaky ReLU se creó para **abordar la desventaja** de la ReLU pura. Al permitir una pequeña pendiente $\alpha$ en el dominio negativo, ayuda a reducir la posibilidad de neuronas muertas.
:::
:::::

<div style="font-size:65%;">

## Tabla Comparativa de Funciones de Activación

| Función        | Ecuación                                                               | Rango       | Ventajas / Desventajas                                                                                     |
|----------------|------------------------------------------------------------------------|-------------|-------------------------------------------------------------------------------------------------------------|
| **Sigmoide**   | $f(x) = \frac{1}{1 + e^{-x}}$                                          | (0, 1)      | Interpretación probabilística; sufre saturación y vanishing gradient.                                       |
| **tanh**       | $f(x) = \tanh(x)$                                                      | (-1, 1)     | Zero-centered; parecido a sigmoide, pero también saturable.                                                 |
| **ReLU**       | $f(x) = \max(0, x)$                                                    | [0, ∞)      | Eficiente, no satura en la región positiva; puede generar “neurona muerta”.                                 |
| **Leaky ReLU** | $f(x) = \begin{cases} x, & x \ge 0 \\ \alpha x, & x < 0 \end{cases}$   | $\mathbb{R}$ | Evita neuronas muertas; pierde parte de la “sparsidad”.                                                    |
| **SoftPlus**   | $f(x) = \ln(1 + e^x)$                                                  | (0, ∞)      | Versión “suave” de ReLU; no tiene discontinuidad en $x=0$; puede saturarse para valores muy negativos.      |
| **GELU**       | $f(x) = x \,\Phi(x)$, donde $\Phi$ es la CDF de la Normal estándar      | $\mathbb{R}$ | Similar a ReLU, pero introduce un componente probabilístico; transición más suave alrededor de $x=0$.      |

</div>

---

> [**Ver código en GoogleColab Funciones de activación**](https://colab.research.google.com/drive/1hLj81KfVSG-yfRn1hxEFgq2dmJoFJtTQ?authuser=1#scrollTo=7596195b-efaa-4ca3-99aa-cd81339012b3)

# Funciones de Salida: Softmax

## Funciones de Salida para Clasificación Multiclase

::::: {.columns style="font-size:75%;"}
::: {.column width="60%"}
-   **Uso principal:** Clasificación multiclase (K clases).\

-   **Definición:** $$
    \text{softmax}(s_i) = \frac{e^{s_i}}{\sum_{j=1}^{K} e^{s_j}}
    $$

-   **Interpretación:**

    -   Transforma un vector de puntajes $(s_1, s_2, \ldots, s_K)$ en probabilidades que **suman 1**.\
    -   Resalta la clase de mayor puntaje y “suprime” las restantes, haciendo que una de las probabilidades destaque.
:::

::: {.column width="40%"}
![Figura 4.7. Comparación de salidas crudas vs. softmax](presentacion_files/34.png) **Adicional**\
Las probabilidades suman 1 para que haya una distribución de probabilidad coherente sobre las $K$ clases posibles.
:::
:::::

# Multi-layer Perceptron (MLP)

El Multi-layer Perceptron crea efectivamente una representación de características jerárquica y puede manejar datos no linealmente separables. Empecemos resolviendo el problema XOR usando un MLP.

## Resolviendo el Problema XOR (I)

::::: {.columns style="font-size:75%;"}
::: {.column width="45%"}
Consideremos una compuerta XOR. Ya hemos visto que no puede implementarse con un SLP. También se puede crear una compuerta NAND de la misma manera que una compuerta AND con entradas negadas.
:::

::: {.column width="55%"}
![Figura 4.8. Placeholder: NAND y OR con SLP](presentacion_files/36.png) Ahora, consideremos una red con dos entradas $X_1$ y $X_2$. Es sencillo crear un SLP para la implementación de las compuertas NAND y OR, tal como se muestra en la Figura.
:::
:::::

## Resolviendo el Problema XOR (II)

::::: {.columns style="font-size:65%;"}
::: {.column width="45%"}
Para construir una compuerta XOR, la salida de las redes anteriores actúa como entrada para una neurona en la siguiente capa, la cual implementa la compuerta AND Figura.

**Matemática de XOR**

-   XOR puede representarse por la expresión booleana: $$
      Y = AB' + A'B 
      $$

-   **NAND** puede representarse como: $$
      Y = (A \cdot B)'
      $$

-   lo cual, aplicando la Ley de De Morgan, se escribe como: $$
      Y = A' + B'
      $$
:::

::: {.column width="55%"}
![Figura 4.9. Placeholder: XOR con NAND, OR y AND](presentacion_files/37.png)

-   Ahora, multiplicando $A + B$ con $Y$, obtenemos: $$
      Z = (A + B)(A' + B') = AA' + AB' + BA' + BB'
      $$ $$
      Z = AB' + A'B
      $$ Por tanto, XOR puede verse como AND de NAND y OR.
:::
:::::

## Arquitectura de MLP y Forward Pass (I)

::::: {.columns style="font-size:70%;"}
::: {.column width="45%"}
**Architecture of MLP and Forward Pass**\
Un Multi-layer Perceptron tiene una capa de entrada, una de salida y al menos una capa oculta.

**Cálculo en la capa oculta**\
Sea $X_1, X_2, \dots, X_n$ la entrada y $W_{ij}$ los pesos entre la primera y la segunda capa. Para la neurona $p$ en la capa oculta:

$$
U_p = \sum_i X_i \,W_{ip} + b_p
$$

La salida de esa neurona: $$
V_p = f(U_p)
$$
:::

::: {.column width="55%"}
![Figura 4.10. Placeholder: MLP con n entradas y 1 salida](presentacion_files/38.png)
:::
:::::

------------------------------------------------------------------------

## Arquitectura de MLP y Forward Pass

**Cálculo en la capa de salida**\
Sea (q) la neurona en la capa de salida:

$$
U_q = \sum_p V_p \,W_{pq} + b_q
$$

$$
V_q = f(U_q)
$$

En cada capa, calculamos la salida para usarla como entrada de la siguiente capa. Este tipo de red se denomina **Feed-Forward Network**.

------------------------------------------------------------------------

::::: {.columns style="font-size:70%;"}
::: {.column width="50%"}
**Como ejemplo**, consideremos la clasificación de un subconjunto del **IRIS dataset** (dos clases: Setosa y Versicolor, cuatro features). La red tiene cuatro neuronas de entrada, dos en la capa oculta y una en la de salida.

-   **Forward Pass**\
    Se asume que los pesos y bias están inicializados. Para la primera capa oculta: ![Figura 4.10. Cálculos](presentacion_files/40.png) La salida de la neurona final: $$
    O = f\bigl(V_1 \, w_{1}^2 + V_2 \, w_{2}^2 + b\bigl)
    $$
:::

::: {.column width="50%"}
![Figura 4.11. Placeholder: MLP con 4 entradas, 2 oculta, 1 salida](presentacion_files/39.png)
:::
:::::

## Cálculo de la Salida (Ejemplo IRIS)

**Comparación y Error**\
Una vez que se calcula la salida $O$, se compara con la salida deseada $\hat{y}$ (o $y_i$, etc.). Por ejemplo, si usamos error cuadrático:

$$
L = \frac{1}{2}(\hat{y} - O)^2.
$$

**Idea:** Minimizar $L$. Para ello, es necesario ajustar los pesos mediante un método de optimización, típicamente **Gradiente Descendiente**.

# Modularidad en un Sistema de Deep Learning

Según la perspectiva presentada en el complemento, un sistema de deep learning se compone de:

-   **Bloques de representación:** Secuencias de transformaciones lineales y no lineales que extraen características relevantes.\
-   **Clasificador lineal:** Una última capa que asigna la etiqueta de clase basándose en las representaciones aprendidas.

------------------------------------------------------------------------

![Figura 4.7. Sistema de Deep Learning](presentacion_files/33.png)

> Los módulos se ensamblan en grafos y se optimizan mediante métodos basados en gradiente

# Optimización: Propagación y Gradiente Descendente

## Gradiente Descendiente

::::: {.columns style="font-size:75%;"}
::: {.column width="60%"}
En un pipeline de Machine Learning, se suelen:

1.  Preprocesar datos.\
2.  Extraer y seleccionar características.\
3.  Hacer predicciones con un modelo.\
4.  Diseñar una función de pérdida para medir la diferencia entre predicción y valor real.
:::

::: {.column width="40%"}
![Figura 4.7. gráfico en 3D ilustra cómo se va minimizando la función de pérdida a lo largo de dos parámetros](presentacion_files/35.png)
:::
:::::

------------------------------------------------------------------------

El proceso de aprendizaje de una red neuronal se divide en dos fases:

1.  **Forward Propagation:**\
    Se calculan las salidas pasando la entrada a través de todas las capas.

2.  **Backward Propagation:**\
    Se calcula el gradiente del error para ajustar los parámetros mediante técnicas de optimización **gradiente descendente**.

------------------------------------------------------------------------

La regla de actualización de parámetros es:

$$
  \theta := \theta - \eta\, \nabla_\theta \mathcal{L}(\theta)
  $$

donde:

-   $\theta$ representa los parámetros (pesos y bias).\
-   $\eta$ es la tasa de aprendizaje.\
-   $\nabla_\theta \mathcal{L}(\theta)$ es el gradiente del error.

------------------------------------------------------------------------

[**Consulta el notebook 03-Gradient_Descent.ipynb para un ejemplo interactivo.**](https://colab.research.google.com/drive/1qnfi3gbIPyBGRDmhpzVCHKgQA11zlqLw?authuser=1#scrollTo=_LGPUWenkH2A)

## Descenso por Gradiente

1.  **Valor predicho**\
    $$
    \hat{y} = f(\mathbf{W}^T \mathbf{X} + b)
    $$

2.  **Función de pérdida** (mitad del error cuadrático)\
    $$
     \frac{\partial L}{\partial \mathbf{W}} = \dfrac{\partial\bigl(1/2(\hat{y}_i - y_i)^2\bigr)}{\partial \mathbf{W}}
    $$

------------------------------------------------------------------------

3.  **Derivada de la pérdida respecto a (W)**\
    $$
    \frac{\partial L}{\partial \mathbf{W}}=
      \dfrac{\partial\bigl(1/2(f(\mathbf{W}^T \mathbf{X} + b) - y_i)^2\bigr)}{\partial \mathbf{W}}
    $$

4.  **Derivada de la pérdida respecto a (b)**\
    $$
    \frac{\partial L}{\partial \mathbf{W}}=
      \dfrac{\bigl(f(\mathbf{W}^T \mathbf{X} + b) - y_i)\bigr)\times f(1-f)\times \mathbf{X} }{\partial \mathbf{W}}
    $$

------------------------------------------------------------------------

**Actualización de los pesos y bias**\
$$
\mathbf{W}_\text{nuevo}
=
\mathbf{W}_\text{viejo}
-
\alpha
\;\frac{\partial L}{\partial \mathbf{W}}
,\qquad \\
b_\text{nuevo}
=
b_\text{viejo}
-
\alpha (\hat{y}_i - y_i)f(1-f).
$$

**Comentario:**\
- $\alpha$ = tasa de aprendizaje\
- $\mathbf{X}$ = vector de entrada


## Backpropagation (I)

::: {style="font-size:90%;"}
# Backpropagation y Ajuste de Pesos  

En redes neuronales multicapa (MLP), el **backpropagation** es el método que ajusta los pesos en **todas** las capas, propagando el error desde la salida hasta la capa de entrada.

:::

## Explicación Conceptual de Backpropagation

::: {style="font-size:80%;"}
**Backpropagation** es el proceso para actualizar los pesos en cada capa de una MLP:

1. **Cálculo de Error** en la capa de salida.  
2. **Propagación** de ese error hacia capas anteriores.  
3. **Regla de Actualización** de los pesos mediante descenso por gradiente.

   La lógica se basa en la **regla de la cadena (chain rule)**:

-  Cada neurona recibe parte de la “responsabilidad” del error total.  
-  Las capas ocultas se actualizan después de que la capa de salida ya calculó su contribución al error.
:::

---

## Algoritmo de Backpropagation

::: {style="font-size:80%;"}
El **algoritmo en 7 pasos**. En resumen:

1. **Inicializar** los pesos y bias con valores pequeños aleatorios.  
2. **Forward Pass**: calcular sumas ponderadas y activaciones hasta la salida.  
3. **Calcular el Error** (p. ej., error cuadrático).  
4. **Calcular gradiente** en la capa de salida.  
5. **Actualizar** los pesos de la última capa.  
6. **Propagar** el error a la(s) capa(s) oculta(s) y actualizar sus pesos.  
7. **Repetir** (Forward + Backward) hasta convergencia o hasta un máximo de etapas.

A continuación detallamos cada paso con sus ecuaciones.
:::

## Pasos 1-2

::: {style="font-size:75%;"}
### **Paso 1: Inicialización**
- Se asignan valores aleatorios pequeños a $W_{ij}$ y a los bias $b_j$ de todas las capas.

### **Paso 2: Forward Pass**
- Para cada capa $k$, calculamos la suma ponderada:  
  $$
  U_j^{(k)} = \sum_i \Bigl(\mathbf{X}_iW_{ij}^{(k)} \,\Bigr) + b_j^{(k)} 
  $$  
  y luego la salida de cada neurona:  
  $$
  O_j^{(k)} = f\bigl(U_j^{(k)}\bigr).
  $$
:::
## Pasos 3 al 4
::: {style="font-size:80%;"}
### **Paso 3: Calcular la función de error**
 
  $$
  L = \frac{1}{2}\bigl(\hat{y}_i - y_i\bigr)^2.
  $$
  (La constante $1/2$ facilita la derivada.)

### **Paso 4: Calcular el gradiente en la capa de salida**

  $$
  \frac{\partial L}{\partial \mathbf{W}} = \dfrac{\partial\bigl(1/2(\hat{y}_i - y_i)^2\bigr)}{\partial \mathbf{W}}.
  $$
:::
## Pasos 5 al 6
::: {style="font-size:70%;"}
### **Paso 5: Actualizar los pesos de la última capa**
- La regla de descenso por gradiente:  
  $$
  \mathbf{W}_\text{nuevo}=\mathbf{W}_\text{viejo}-\alpha\;\frac{\partial L}{\partial \mathbf{W}}
  $$
  
### **Paso 6: Actualizar los pesos de la(s) capa(s) oculta(s) (Backpropagation)**
1. Para cada neurona $p$ en la capa $(k-1)$:  
   
   $$
  W_{ij}^{(k)}= W_{ij}^{(k)} - \eta\;\delta_i^{(k)}\; O_j^{(k-1)},
  $$
   
    donde $\eta$ es la tasa de aprendizaje y $O_i^{(k-1)}$ la salida de la neurona $i$ en la capa anterior. 

:::

---

::: {style="font-size:70%;"}
2. Actualizar pesos:  
   $$
   \delta_{i}^{k}=O_i^{k}(1-O_i^{k})\sum_{j=1}^{M_{k+1}} \partial_j^{(k+1)} W_{ij}^{(k+1)}
   \\
   \partial_{j}^{k}=O_j^{(k+1)}(1-O_j^{(k+1)})(O_j^{(k+1)}-y_j).
   $$


### **Paso 7: Repetir**
- Se repite **forward pass + backward pass** por múltiples etapas, hasta convergencia o hasta el número máximo de iteraciones.
:::

## Presentación del Taller semana 2
::: {style="font-size:80%;"}
# Taller Práctico: Entrenando MLP con scikit-learn y Keras

En este taller cubriremos:

1. **Clasificación binaria** usando **Wine Dataset** (scikit-learn).  
2. **Clasificación binaria** con **Breast Cancer Dataset** (Keras).  
3. Análisis de **número de neuronas**, **capas ocultas** y **tasa de aprendizaje**.

Al final, tendremos una **comprensión práctica** de cómo configurar **Multi-Layer Perceptrons** y cómo los cambios en la arquitectura impactan la precisión y la pérdida.
:::

## Ejercicio 1 - Wine Dataset (scikit-learn)

::: {style="font-size:80%;"}

**Objetivo**: Clasificar las dos primeras clases del dataset Wine (13 características, 130 muestras), y comparar dos MLP:

-  Uno con **parámetros por defecto** (100 neuronas en la capa oculta).
-  Otro con solo **3 neuronas** en la capa oculta.

### Pasos Generales

1. **Cargar** el dataset y filtrar las clases a usar.  
2. **Separar** datos en entrenamiento y prueba (`train_test_split`).  
3. **Definir** y **entrenar** dos modelos MLP (uno por defecto, otro con 3 neuronas).  
4. **Comparar** precisión en el set de prueba.

[**Ver código en GoogleColab Ejercicio 4_1**](https://colab.research.google.com/drive/1nnMZpwhRmqSus3_thtvy0ZQceS5V85Bz?authuser=1)


:::


## Conclusiones Ejercicio 1

::: {style="font-size:80%;"}

1. **Más neuronas** → mayor capacidad de ajuste → tiende a mejor desempeño (aunque no siempre).  
2. Un número muy bajo de neuronas puede **subajustar** (underfitting).  
3. Elección del tamaño de la capa oculta = **hiperparámetro** que depende del dataset y se suele ajustar de forma **empírica**.

### **Tarea Sugerida**:
- Prueba distintos tamaños de capa oculta: (10,), (50,), (200,).  
- Observa cómo varía la precisión de prueba y el tiempo de entrenamiento.
:::


## Ejercicio 2 - Breast Cancer (Keras)

::: {style="font-size:70%;"}

**Objetivo**: Utilizar la librería **Keras** para crear un MLP que clasifique el dataset de **Cáncer de Mama** (Breast Cancer).  
- Modelo 1: 1 capa oculta (16 neuronas).  
- Modelo 2: 2 capas ocultas (16 y 8 neuronas).

Además, **experimentar** con optimizadores (SGD, RMSprop, Adam) y tasas de aprendizaje.

### Pasos Generales
1. Cargar `load_breast_cancer` (569 muestras, 30 features).  
2. Separar en entrenamiento y prueba (`train_test_split`).  
3. Definir un **Sequential** con `Dense(...)`.  
4. Compilar el modelo (`model.compile(...)`) indicando el optimizador, la pérdida (`binary_crossentropy`) y la métrica (`accuracy`).  
5. Entrenar (`model.fit(...)`) y evaluar (`model.evaluate(...)`).

:::


## Código Ejercicio 2 (Keras, 1 capa oculta)

[**Ver código en GoogleColab Ejercicio 4_2**](https://colab.research.google.com/drive/1x3oNyYN2n6nFkhVyQc_uy2KkHLDYbO_8?authuser=1)


## Visualización de Resultados

::: {style="font-size:80%;"}
Generalmente, se grafican:

-  **Training vs. Validation Loss**  
-  **Training vs. Validation Accuracy**


Podemos ver si el modelo **sobreeentrena** (training accuracy sube y validation no mejora) o si converge adecuadamente.
:::


## Múltiples Capas Ocultas y Optimización

::: {style="font-size:80%;"}
**Modelo 2**: 2 capas ocultas (16 y 8 neuronas).  

-   [**Ver código en GoogleColab Ejercicio 4_3**](https://colab.research.google.com/drive/1p05uOEd0rSPrF4aT_weNLlIhCyggzjIN?authuser=1)
-   [**Ver código en GoogleColab Ejercicio 4_4**](https://colab.research.google.com/drive/1T5g-BQUXDKA4IiwhVLd5TVGoELxwwwHQ?authuser=1#scrollTo=2R4aP9UiWkmO)
-   [**Ver código en GoogleColab Ejercicio 4_5**](https://colab.research.google.com/drive/1RIVBByXg7LgwURINTE6_1AJoKtdUKvmJ?authuser=1#scrollTo=FZEXdBM6j0a2)
-   [**Ver código en GoogleColab Ejercicio 4_6**](https://colab.research.google.com/drive/1cUgSQK6ELHRArIFZsG8ud5CPrBW_mdsq?authuser=1)

        

**Cambiar optimizador / LR** (ej. Adam con lr=0.001):


**Observa** la variación de desempeño con distintas capas y optimizadores.
:::


## Tarea Final

::: {style="font-size:70%;"}
1. **Experimentar con más etapas**: aumenta o reduce `epochs` (por ejemplo, 100, 200).  
2. **Cambiar la función de activación**: `'relu'` en lugar de `'sigmoid'` entre otras.  
3. **Variar el número de capas**:  
   - 1 capa oculta (p.ej. 50 neuronas).  
   - 2 capas ocultas (p.ej. 25 y 12).  
4. **Monitorear** la pérdida y precisión en cada experimento.  
5. **Comparar**: ¿Cuál configuración converge más rápido? ¿Cuál obtiene mejor exactitud?

Al final, **documenta** tus hallazgos:
- ¿Qué impacto tuvo la tasa de aprendizaje?  
- ¿Cambió el problema de *overfitting*?

¡Éxitos en tu exploración con MLPs!
:::

# Conclusiones y Debate

::: {style="font-size:80%;"}
-   Hemos explorado desde la base del perceptrón y su inspiración biológica hasta la complejidad de sistemas de deep learning.\
-   Se ha destacado la importancia de las transformaciones no lineales para lograr la separación de datos complejos.\
-   La optimización mediante propagación hacia atrás y gradiente descendente es fundamental para ajustar los parámetros.
-  **Número de neuronas** y **capas ocultas** son hiperparámetros cruciales.  
-  **Tasa de aprendizaje** y **optimizador** pueden alterar la velocidad y estabilidad de convergencia.  
- Para problemas reales, conviene **probar múltiples configuraciones** (grid search o random search), con validación cruzada.  

:::

# Referencias

::: {style="font-size:80%;"}
1.  Definición informal de ML.\
2.  Definición de IA.\
3.  Definición formal de ML (Tom Mitchell).\
4.  Minsky, M. & Papert, S. (1969). *Perceptrons*. MIT Press.\
5.  Rumelhart, D. E., Hinton, G. E., & Williams, R. J. (1986). *Learning representations by back-propagating errors*. Nature, 323(6088), 533–536.\
6.  LeCun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). *Gradient-based learning applied to document recognition*. Proceedings of the IEEE, 86(11), 2278–2324.\
7.  Hubel, D. H., & Wiesel, T. N. (1962). *Receptive fields, binocular interaction and functional architecture in the cat's visual cortex*. Journal of Physiology, 160, 106–154.\
8.  Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). *ImageNet classification with deep convolutional neural networks*. NeurIPS.\
9.  He, K., Zhang, X., Ren, S., & Sun, J. (2016). *Deep residual learning for image recognition*. CVPR.\
10. Material y figuras extraídos de: *Complemento Capítulo 3: Introduction to Deep Learning* :contentReference[oaicite:6]{index="6"}.

© Harsh Bhasin 2024\
H. Bhasin, *Hands-on Deep Learning*, [DOI:10.1007/979-8-8688-1035-0_1](https://doi.org/10.1007/979-8-8688-1035-0_1) \`\`\`

:::